## Summarization

In [25]:
import os, pickle

response_path = ".summarization_backup/"

In [30]:
# crear una funcion que reciba un string y formatee el texto para que sea legible
# utilizar un maximo de 50 caracteres por linea

def format_text(text):
    formatted_text = ""
    for i in range(0, len(text), 50):
        formatted_text += text[i:i+50] + "\n"
    return formatted_text


In [32]:
# Primer run: response_1709556534.pkl
# documento index = 0
# pagina index = 0
# prompt_template = """Generar un resumen que capture los patrones relevantes\
#  en los datos del TEXTO. La respuesta debe ser de 1000 caracteres como maximo.
# TEXTO:
# "{text}"
# RESUMEN:"""

with open(response_path + "response_1709556534.pkl", "rb") as f:
    response = pickle.load(f)

print(format_text(response))

La póliza de accidentes personales/reembolso de ga
stos médicos establece que la compañía aseguradora
 reembolsará al asegurado los gastos médicos razon
ables y acostumbrados incurridos durante la vigenc
ia de la póliza, siempre y cuando se cumplan ciert
os requisitos. Estos requisitos incluyen que el ac
cidente ocurra durante la vigencia de la póliza, q
ue los gastos se realicen dentro del período de du
ración de reembolso, que el monto de gastos supere
 el deducible establecido, que la suma de los reem
bolsos no supere el monto máximo establecido, que 
los gastos no sean excluidos y que las prestacione
s médicas sean entregadas por los prestadores indi
cados. Los reembolsos se realizarán hasta el monto
 máximo establecido y de acuerdo a los términos y 
límites establecidos en las condiciones particular
es de la póliza. Además, se establece que el reemb
olso será complementario a lo que cubra el sistema
 previsional o de salud al que esté afiliado el as
egurado. En caso de contar con 

In [36]:
# Segundo run: response_1709557379.pkl
# documento index = 0
# pagina index = 0
# prompt_template = """Extraer los bullet-points de los articulos\
#  contenidos en el TEXTO. La respuesta debe ser de 1000 caracteres como maximo.
# TEXTO:
# "{text}"
# RESUMEN:"""

with open(response_path + "response_1709557379.pkl", "rb") as f:
    response = pickle.load(f)
    
print(response["output_text"])


- El contrato de seguro se regirá por las disposiciones legales y las estipulaciones más beneficiosas para el asegurado o beneficiario.
- La cobertura de la póliza incluye el reembolso de los gastos médicos razonables y acostumbrados incurridos por el asegurado en caso de accidente.
- Los gastos deben ser realizados en los prestadores de salud determinados por la aseguradora y durante el período de duración del reembolso.
- Se establecen requisitos para el reembolso de los gastos, como que el accidente haya ocurrido durante la vigencia de la póliza, que se haya incurrido en los gastos dentro del período de duración de reembolso, que el monto de gastos supere el deducible establecido, entre otros.
- Los reembolsos están sujetos a límites y topes establecidos en las condiciones particulares de la póliza, así como a porcentajes de bonificación o reembolso entregados por el sistema de salud previsional al que esté afiliado el asegurado.
- La aseguradora reembolsará los gastos médicos como 

In [37]:
print(len(response["output_text"]))

1353


## func 1

In [1]:
import os
from utils.config import DOWNLOAD_PATH


file_n = 8
pdf_files = os.listdir(DOWNLOAD_PATH)
selected_file = os.path.join(DOWNLOAD_PATH, pdf_files[file_n])
pattern = "ART.CULO\s\d+.+:?"


# func 1:
from eda_tools.pdf_file_tools import extract_patterns

print(selected_file.split("\\")[-1])
extract_patterns(selected_file, pattern)


POL320210210.pdf


['ARTICULO 1°: REGLAS APLICABLES AL CONTRATO',
 'ARTÍCULO 2º: COBERTURA',
 'ARTÍCULO 3º: DEFINICIONES.',
 'ARTÍCULO 4º: EXCLUSIONES.',
 'ARTICULO 5º: CARENCIA',
 'ARTÍCULO 6°: OBLIGACIONES DEL ASEGURADO',
 'ARTICULO 7°: DECLARACIONES DEL ASEGURADO',
 'ARTICULO 8º: PRIMAS Y EFECTOS DEL NO PAGO DE LAS PRIMAS',
 'ARTICULO 9°: BENEFICIARIOS',
 'ARTÍCULO 10º: DENUNCIA DE SINIESTROS',
 'ARTICULO 11º: VIGENCIA Y TERMINACIÓN',
 'ARTÍCULO 12º: COMUNICACIÓN ENTRE LAS PARTES',
 'ARTÍCULO 13º: SOLUCIÓN DE CONTROVERSIAS',
 'ARTÍCULO 14°: CLAUSULAS ADICIONALES',
 'ARTÍCULO 15°: DOMICILIO']

## Spliting documents by article

**TRATAMIENTO FILE NUMBER 8**

In [1]:
import os
from eda_tools.pdf_file_tools import extract_patterns
from eda_tools.doc_fragmentation import extract_documents_from_text
from utils.config import DOWNLOAD_PATH

# File 8 
file_n = 8
pdf_files = os.listdir(DOWNLOAD_PATH)

selected_file = os.path.join(DOWNLOAD_PATH, pdf_files[file_n])
pattern = "ART.CULO\s\d+.+:?"
articles = extract_patterns(selected_file, pattern)

documents = extract_documents_from_text(selected_file, articles)

In [2]:
documents

[Document(page_content='ARTICULO 1°: REGLAS APLICABLES AL CONTRATO\nSe aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las\nnormas legales de carácter imperativo establecidas en el Título VIII, del Libro II, del Código de Comercio. Sin\nembargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el\nasegurado o beneficiario.\n \n', metadata={'file': 'POL320210210.pdf', 'policy_header': 'SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO', 'article_title': 'ARTICULO 1°: REGLAS APLICABLES AL CONTRATO'}),
 Document(page_content='ARTÍCULO 2º: COBERTURA\nLa compañía aseguradora reembolsará los gastos médicos razonables, acostumbrados y efectivamente\nincurridos por el asegurado, asociados a un Evento cubierto por esta póliza, en los términos y condiciones\nestablecidas en ésta, siempre que haya transcurrido el periodo de carencia establecido en las Condiciones\nParticulares, que la póliza se encuentre vigente

In [79]:
file_n = 2
selected_file = os.path.join(DOWNLOAD_PATH, pdf_files[file_n])
print(selected_file.split("\\")[-1])
extract_patterns(selected_file, pattern)

POL320150503.pdf


['ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO',
 'ARTÍCULO 2º: COBERTURA Y MATERIA ASEGURADA',
 'ARTÍCULO 3°: LIMITACIONES DE LAS COBERTURAS',
 'ARTÍCULO 4º: DEFINICIONES',
 'ARTÍCULO 5º: EXCLUSIONES',
 'ARTÍCULO 6°: OBLIGACIONES DEL ASEGURADO',
 'ARTÍCULO 7°: DECLARACIONES DEL CONTRATANTE Y DEL ASEGURADO',
 'ARTÍCULO 8º: PRIMAS Y EFECTO DEL NO PAGO DE LA PRIMA',
 'ARTÍCULO 9º: DENUNCIA DE SINIESTROS',
 'ARTÍCULO 10°: CALCULO DE LOS GASTOS REEMBOLSABLES',
 'ARTICULO 11°: LIQUIDACION DE LOS GASTOS, FORMA DE PAGO Y MONTO MAXIMO DE GASTOS',
 'ARTÍCULO 12°: APLICACION DEL DEDUCIBLE',
 ' ARTÍCULO 13°: VIGENCIA Y DURACIÓN DEL CONTRATO DE SEGURO',
 'ARTÍCULO 14°: INCORPORACIÓN DE ASEGURADOS E INICIO DE LA VIGENCIA DE LA COBERTURA',
 'ARTÍCULO 15°: TERMINACIÓN DEL CONTRATO',
 ' ARTÍCULO 16º: AJUSTE DE LA PRIMA',
 'ARTÍCULO 17°: MONEDA O UNIDAD DEL CONTRATO',
 'ARTICULO 18°: REHABILITACION DE LA PÓLIZA',
 'ARTÍCULO 19º: CONTRIBUCIONES E IMPUESTOS',
 'ARTÍCULO 20º: COMUNICACIÓN ENTRE LAS PARTES',

# func 2

In [73]:
import os
from utils.config import DOWNLOAD_PATH


file_n = 0
pdf_files = os.listdir(DOWNLOAD_PATH)
selected_file = os.path.join(DOWNLOAD_PATH, pdf_files[file_n])
pattern = "ART.CULO\s\d+.+:"

# func2
from eda_tools.pdf_file_tools import extract_patterns_per_page

results = extract_patterns_per_page(selected_file, pattern, extend=False)
results

[['ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO',
  'ARTÍCULO 2º: COBERTURA Y MATERIA ASEGURADA'],
 [],
 ['ARTÍCULO 3°: LIMITACIONES DE LAS COBERTURAS', 'ARTÍCULO 4º: DEFINICIONES'],
 [],
 [],
 ['ARTÍCULO 5º: EXCLUSIONES'],
 [],
 ['ARTÍCULO 6°: OBLIGACIONES DEL ASEGURADO',
  'ARTÍCULO 7°: DECLARACIONES DEL CONTRATANTE Y DEL ASEGURADO',
  'ARTÍCULO 8º: PRIMAS Y EFECTO DEL NO PAGO DE LA PRIMA'],
 ['ARTÍCULO 9º: DENUNCIA DE SINIESTRO '],
 ['ARTÍCULO 10°: CALCULO DE LOS GASTOS REEMBOLSABLES',
  'ARTÍCULO 11°: LIQUIDACIÓN DE LOS GASTOS, FORMA DE PAGO Y MONTO MÁXIMO DE GASTOS'],
 ['ARTÍCULO 12°: APLICACIÓN DEL DEDUCIBLE',
  'ARTÍCULO 13°: VIGENCIA Y DURACIÓN DEL CONTRATO DE SEGURO'],
 ['ARTÍCULO 14°: INCORPORACIÓN DE ASEGURADOS E INICIO DE LA VIGENCIA DE LA COBERTURA',
  'ARTÍCULO 15°: TERMINACIÓN DEL CONTRATO',
  'ARTÍCULO 16º: AJUSTE DE LA PRIMA'],
 ['ARTÍCULO 17°: MONEDA O UNIDAD DEL CONTRATO',
  'ARTICULO 18°: REHABILITACIÓN DE LA PÓLIZA',
  'ARTÍCULO 19º: CONTRIBUCIONES E IMPUESTOS ',
  '

# Experiment for EnsambleRetriever

In [52]:
# Important for EnsambleRetriever

def distribute_weights(string_list):
    # Calculate the total length of all strings
    total_length = sum(len(string) for string in string_list)
    
    # Calculate the weights of each string
    weights = [len(string) / total_length for string in string_list]
    
    # Normalize the weights so that the sum is 1
    sum_weights = sum(weights)
    normalized_weights = [weight / sum_weights for weight in weights]
    
    return normalized_weights



In [53]:
distribute_weights(results)

[0.09855313092979125,
 0.0,
 0.08467741935483869,
 0.0,
 0.0,
 0.038780834914611,
 0.0,
 0.14622865275142313,
 0.05230075901328272,
 0.1042457305502846,
 0.06973434535104363,
 0.092623339658444,
 0.1979364326375711,
 0.11491935483870964]